In [ ]:
from datasets import load_dataset

dataset = load_dataset("openbookqa", "main", split=["train", "test"])

In [ ]:
def get_prompt(dataset, index, perm=[0, 1, 2, 3], has_choice=False):
    json_line = dataset[index]
    question = json_line["question_stem"]
    choices = json_line["choices"]
    choice_texts = choices["text"]
    choice_texts = [choice_texts[perm[i]] for i in range(len(choice_texts))]
    candidates = " ".join(
        [
            f"({label if has_choice else ' '}) {text}"
            for text, label in zip(choice_texts, choices["label"])
        ]
    ).replace("\n", " ")
    answer_key = json_line["answerKey"]
    answer_key_idx = ord(answer_key[0]) - ord("A")
    answer_text = choices["text"][answer_key_idx]
    question_pad = f"{question} \\n "
    prompt = f"{question_pad}{candidates}"
    return prompt, answer_text

In [ ]:
import itertools

perm_order = list(itertools.permutations([0, 1, 2, 3], 4))

In [53]:
from tqdm import tqdm
import pickle

train_without_abcd = []
for index, ques in enumerate(tqdm(dataset[0])):
    for order in perm_order:
        train_without_abcd.append(
            get_prompt(dataset[0], index, order, has_choice=False)
        )
pickle.dump(train_without_abcd, open("train_without_abcd.pkl", "wb"))

test_without_abcd = []
for index, ques in enumerate(tqdm(dataset[1])):
    for order in perm_order:
        test_without_abcd.append(
            get_prompt(dataset[1], index, order, has_choice=False)
        )
pickle.dump(test_without_abcd, open("test_without_abcd.pkl", "wb"))

  0%|          | 0/4957 [00:00<?, ?it/s]

100%|██████████| 4957/4957 [00:19<00:00, 253.80it/s]


In [52]:
import json
import pickle
obj = pickle.load(open("test_with_abcd.pkl", "rb"))

with open("test_with_abcd.txt", "w") as f:
    json.dump(obj, f, indent=2)


In [3]:
# def get_prompt(index, perm):
#     json_line = dataset["test"][index]
#     question = json_line["question_stem"]
#     choices = json_line["choices"]
#     choice_texts = choices["text"]
#     choice_texts = [choice_texts[perm[i]] for i in range(len(choice_texts))]

#     def change(text, leng=20, is_question=False):
#         pad_x = [0] * leng
#         encoded = tokenizer.encode(text)[:-1]
#         if len(encoded) > leng:
#             print("too long ", len(encoded), leng, is_question)
#         pad_x[: len(encoded)] = encoded
#         return tokenizer.decode(pad_x)

#     candidates = " ".join(
#         [
#             change(f"( ) {text}", ANSWER_PAD)
#             for text, label in zip(choice_texts, choices["label"])
#         ]
#     ).replace("\n", " ")
#     # print(json_line)
#     answer_key = json_line["answerKey"]
#     answer_key_idx = ord(answer_key[0]) - ord("A")
#     # answer_text = choice_texts[answer_key_idx]
#     # id = "OBQA_" + json_line['id']
#     question_pad = f"{question} \\n "
#     prompt = f"{change(question_pad,QUESTION_PAD, is_question=True)}{candidates}"
#     return prompt, choice_texts